In [1]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.style
matplotlib.style.use("seaborn")
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import textwrap
import seaborn as sn
sn.color_palette("hls", 17)
import scipy.stats as st
import math

import pingouin as pg
plt.rcParams["font.family"] = "Times New Roman"

In [2]:
import sys
import os
sys.path.append(os.path.join("..", "..", ".."))

from mika.kd.trend_analysis import *
from mika.utils import Data

C:\Users\srandrad\Anaconda3\lib\site-packages\plotly\graph_objs\__init__.py:288: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(ipywidgets.__version__) >= LooseVersion("7.0.0"):
C:\Users\srandrad\Anaconda3\lib\site-packages\plotly\graph_objs\__init__.py:288: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(ipywidgets.__version__) >= LooseVersion("7.0.0"):
C:\Users\srandrad\Anaconda3\lib\site-packages\plotly\graph_objs\__init__.py:288: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(ipywidgets.__version__) >= LooseVersion("7.0.0"):
C:\Users\srandrad\Anaconda3\lib\site-packages\plotly\graph_objs\__init__.py:288: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(ipywidgets.__version__) >= LooseVersion("7.0.0"):
C:\Users\srandrad\Anaconda3\lib\

In [3]:
list_of_attributes = ["Narrative"]
#file = os.path.join(os.path.abspath(os.path.join(os.getcwd(), os.pardir, os.pardir, os.pardir)),"data","SAFENET","SAFENET_1999_2021.csv")
file = os.path.join("topic_model_results", 'preprocessed_data.csv')
extra_cols = ["Iteration", "Event Start Date", "Event Stop Date", "Incident Name",
              "Incident Number", "Event State", "Event Jurisdiction", "Event Local Unit",
              "Event Position Title", "Event Task", "Event Management Level", "Event Resources", 
              "Event Incident Type", "Event Incident Activity", "Event Incident Stage", "Contributing Factors",
              "Human Factors", "Other Factors", "Narrative", "Immediate Action Taken",  "SAFENET Create Date"
              ]
document_id_col = "ID"
safenet = Data()
safenet.load(file, preprocessed=True, id_col=document_id_col, text_columns=list_of_attributes, preprocessed_kwargs={'drop_short_docs':False})
df = safenet.data_df
print(len(df))
df = df.dropna(how='all', subset=['Narrative']).reset_index(drop=True)
print(len(df))

2375
2375


In [4]:
df['Year'] = [df.at[i,'Event Start Date'].split('/')[-1].split(" ")[0] for i in range(len(df))]
df['Day'] = [df.at[i,'Event Start Date'].split('/')[1] for i in range(len(df))]
df['Month'] = [df.at[i,'Event Start Date'].split('/')[0] for i in range(len(df))]

In [5]:
df['ID'] = [str(int(d)) for d in df['ID']]

In [6]:
hazard_file = os.path.join('topic_model_results','hazard_interpretation.xlsx')
results_file = os.path.join('topic_model_results', 'lda_results.xlsx')

In [7]:
df

,ID,Iteration,Event Start Date,Event Stop Date,Incident Name,Incident Number,Event State,Event Jurisdiction,Event Local Unit,Event Position Title,...,Contributing Factors,Human Factors,Other Factors,Narrative,Immediate Action Taken,SAFENET Create Date,Raw_Narrative,Year,Day,Month
0,24587,2.0,01/01/1999 00:00,,QUESTIONABLE QUALIFICATIONS,NaN,ND,FWS,NaN,Concerned Firefighter,...,Human Factors;,Decision Making;Leadership;,Poor decision making by qualifying Offical,"[job, many, dept, happen, previously, structur...",Review suspicious qualification. Review qualif...,01/26/2005 00:00,Fast tracking an under qualified person to ove...,1999,01,01
1,24109,2.0,12/09/2000 13:30,,PILE BURNING,NaN,NM,USFS,NaN,For. Tech.,...,Environmental;,NaN,"Factors included very poor road conditions, un...","[drive, however, area, tree, strike, control, ...",If I had been going slower the conditions woul...,12/11/2000 00:00,"Weather was overcast, windy, and cold with occ...",2000,09,12
2,24107,2.0,10/21/2000 00:00,,PARADISE FIRE,C898,AZ,FWS,NaN,Firefighter,...,Communications;Environmental;,NaN,NaN,"[launch, afternoon, violate, violation, aggres...",Do not know what is appropriate action. Do no...,10/29/2000 00:00,"Weather - during the incident we had rain, win...",2000,21,10
3,24105,2.0,10/05/2000 13:50,,NaN,NaN,MT,USFS,Bitteroot NF,Safety Officer,...,Environmental;,NaN,NaN,"[caught, fall, sunny, attach, area, start, log...",- Tighten straps around legs.\r\n\r\n- Prep th...,10/05/2000 00:00,"Mostly sunny, cool, moderate slope, severely b...",2000,05,10
4,24103,2.0,09/27/2000 15:20,,FALCON,NaN,MN,Other,Stacy VFD,Volunteer Firefighter,...,Fire Behavior;Environmental;,NaN,Windy with flashy fuels,"[caught, area, green, look, suffer, respond, s...",The IA crew should have kept closer together a...,10/03/2000 00:00,heavily wooded with grass and peat. Still look...,2000,27,09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2370,45147,2.0,03/16/2021 08:00,03/19/2021 10:30,Dispatch Radio Kiosk kicked off the FS Network,NaN,CO,USFS,Supervisor's Office,Dispatch Center Manager,...,Communications;Equipment;,NaN,NaN,"[ability, previous week, log, busy, kick, uncl...",Dispatch called Duty Officer's to advise of ra...,04/02/2021 13:16,The weekend of March 13-14 there was a record ...,2021,16,03
2371,45367,3.0,03/08/2021 19:13,03/09/2021 17:00,Melder 1 Fire,KIF-32,LA,USFS,Calcasieu Ranger District,NaN,...,Equipment;Environmental;Human Factors;Other;,Decision Making;Leadership;Risk Assessment;Per...,Evacuation of Forest Recreators,"[rider, belief, new, assignment, left, trail, ...",1-Clarification of R8 policy needs to be made ...,05/19/2021 14:58,On March 9th 2021 multiple units from around K...,2021,08,03
2372,45067,2.0,02/02/2021 14:30,02/02/2021 15:00,CDL Random Drug Test,NaN,CA,USFS,NaN,NaN,...,Other;,NaN,Random CDL Drug Test,"[test, walk, extreme, lobby, area, epicenter, ...","Proper PPE worn. N-95 mask, and medical gloves.",02/11/2021 16:01,I was issued a random mandatory CDL drug test ...,2021,02,02
2373,45027,2.0,01/20/2021 12:00,,RxB project,NaN,NJ,State,NaN,Unknown,...,Equipment;Other;,Decision Making;Leadership;Risk Assessment;Per...,normalization of deviance,"[lane, traffic, face, shelter, footwear, way, ...",Report.,01/22/2021 15:05,No shelter<br/>Improper footwear<br/>No traffi...,2021,20,01


In [8]:
def get_hazard_doc_ids(nums, results, results_text_field, docs, doc_topic_distribution, text_field, topic_thresh, preprocessed_df, id_field):
    ids_df = results[results_text_field].loc[nums]
    ids_ = "; ".join(ids_df['documents'].to_list())
    ids_ = [id_ for id_ in docs if id_ in ids_]   
    # ids_ = ids_ only if topic nums > thres
    if doc_topic_distribution is not None:
        new_ids = []
        for id_ in ids_:
            #check that topic prob> thres for at least one num
            id_df = doc_topic_distribution.loc[doc_topic_distribution['document number']==id_].reset_index(drop=True)
            print(len(id_df))
            probs = [float(id_df.iloc[0][text_field].strip("[]").split(" ")[num].strip("\n")) for num in nums]
            max_prob = max(probs)
            if max_prob > topic_thresh:
                new_ids.append(id_)
        ids_ = new_ids
    temp_df = preprocessed_df.loc[preprocessed_df[id_field].astype(str).isin(ids_)].reset_index(drop=True)
    return temp_df, ids_

In [9]:
frequency, docs, hazard_words_per_doc, topics_per_doc, hazard_topics_per_doc = identify_docs_per_hazard(hazard_file, df, results_file, 'Narrative', 'Year', "ID", 'Narrative', doc_topic_dist_field=None)

100%|██████████████████████████████████████████████████████████████████████████████████| 29/29 [00:06<00:00,  4.79it/s]


In [10]:
def create_table(docs, frequency, preprocessed_df, id_field, categories, subcategories, hazards, time_field):#, accuracy_file):
    table = pd.DataFrame({"Category": categories, "Subcategory": subcategories, "Hazards": hazards})
    #accuracy_df = pd.read_excel(accuracy_file, sheet_name=['Summary'])['Summary']
    time_period = preprocessed_df[time_field].unique()
    severities = {name:{str(time_p):[] for time_p in time_period} for name in hazards}
    rates = {name:{str(time_p):0 for time_p in time_period} for name in hazards}
    total_docs_per_year = preprocessed_df[time_field].value_counts()
    total_rates = {hazard:0 for hazard in hazards}
    total_hazard_freq = {hazard:0 for hazard in hazards}
    total_severities_hazard = {hazard:0 for hazard in hazards}
    total_docs = len(preprocessed_df)
    #precision = []
    for hazard in hazards:
        #hazard_precision = accuracy_df.loc[accuracy_df['Hazards']==hazard].reset_index(drop=True)['Accuracy'].tolist()[0]
        #precision.append(round(hazard_precision,3))
        for year in docs[hazard]:
            year = str(year)
            for doc in docs[hazard][year]:
                doc_df = preprocessed_df.loc[preprocessed_df[id_field] == doc].reset_index(drop=True)
                #severities[hazard][year].append(safecom_severity(doc_df.iloc[0]['Persons Onboard'], doc_df.iloc[0]['Injuries'], doc_df.iloc[0]['Damages']))
            rates[hazard][year] = frequency[hazard][year]/total_docs_per_year[year]
            total_hazard_freq[hazard] += frequency[hazard][year]
        total_rates[hazard] = round(total_hazard_freq[hazard]/len(time_period), 3)
        #total_severities_hazard[hazard] = round(np.average([sev for year in severities[hazard] for sev in severities[hazard][year]]),3)
    table["Frequency"] = [total_hazard_freq[hazard] for hazard in total_hazard_freq]
    table["Rate"] = [total_rates[hazard] for hazard in total_rates]
    #table["Severity"] = [total_severities_hazard[hazard] for hazard in total_severities_hazard]
    #table["Precision"] = precision
    return table#, severities, rates

In [11]:
category_fields = ['Contributing Factors', 'Human Factors', 'Other Factors']

In [12]:
def get_categories_from_docs(docs, preprocessed_df, id_field, category_fields = ['Contributing Factors', 'Human Factors', 'Other Factors']):
    categories = {hazard: [] for hazard in docs}
    sub_categories = {hazard: [] for hazard in docs}
    for hazard in docs:
        ids = [id_ for year in docs[hazard] for id_ in docs[hazard][year]]
        hazard_df = preprocessed_df.loc[preprocessed_df[id_field].isin(ids)].reset_index(drop=True)
        #main category is the category with the most reports
        cat_counts = {}
        for cat in category_fields:
            cat_counts[cat] = len(hazard_df.loc[hazard_df[cat].notna()])
        main_category = max(cat_counts, key=cat_counts.get).strip(";")
        categories[hazard] = main_category
        #subcategories are the most common value in that catgeory
        if len(hazard_df[main_category].value_counts()) == 0:
            sub_categories[hazard] = ""
        else:
            sub_cats = hazard_df[main_category].tolist()
            sub_cats = [cat for sub_cat in sub_cats for cat in str(sub_cat).split(", ")]
            sub_cats =[cat for cat in sub_cats if cat != 'nan']
            sub_cat_series = pd.Series(sub_cats).dropna()
            #sub_cat_series = sub_cat_series.loc[sub_cat_series.notna()].reset_index(drop=True)
            #print(sub_cat_series.value_counts())
            sub_cat = sub_cat_series.value_counts().idxmax(axis = 0).strip(";")
            if sub_cat in category_fields:
                categories[hazard] = sub_cat
                sub_cats = hazard_df[sub_cat].tolist()
                sub_cats = [cat for sub_cat in sub_cats for cat in str(sub_cat).split(", ")]
                sub_cats =[cat for cat in sub_cats if cat != 'nan']
                sub_cat_series = pd.Series(sub_cats).dropna()
                sub_cat = sub_cat_series.value_counts().idxmax(axis = 0).strip(";")
            sub_categories[hazard] = sub_cat#hazard_df[main_category].value_counts().idxmax(axis = 0)#list(hazard_df[main_category].value_counts())
            #sub_categories[hazard] = hazard_df[main_category].value_counts().idxmax(axis = 0)
            #print(hazard_df[main_category].value_counts())
    return categories, sub_categories

In [13]:
cat, sub_cat = get_categories_from_docs(docs, df, 'ID')
cat = list(cat.values())
sub_cat = list(sub_cat.values())

In [14]:
#cat, sub_cat

In [15]:
hazards = list(docs.keys())

In [16]:
tab = create_table(docs, frequency, df, "ID", cat, sub_cat, hazards, 'Year')

In [17]:
topic_focused_sheet = pd.read_excel(hazard_file, sheet_name=['topic-focused'])['topic-focused'].set_index('Hazard name')
manual_cats = [topic_focused_sheet.at[hazard, "Hazard Category"] for hazard in tab['Hazards'].tolist()]
tab['Manual Category'] = manual_cats
tab

,Category,Subcategory,Hazards,Frequency,Rate,Manual Category
0,Contributing Factors,Equipment,Fuel Leak,95,4.130,Equipment
1,Human Factors,Decision Making;Leadership;Risk Assessment;Sit...,Food Supply Issues,65,2.826,Human Factors
2,Human Factors,Situational Awareness,Burn,63,2.739,Medical
3,Contributing Factors,Environmental;Human Factors,Dehydration,15,0.652,Medical
4,Human Factors,Fatigue,Heat Exhaustion,17,0.739,Medical
5,Human Factors,Decision Making;Leadership;Risk Assessment;Per...,Insufficient PPE,143,6.217,Equipment
6,Human Factors,Decision Making;Leadership;Risk Assessment;Per...,Inaccurate Weather Data,16,0.696,Human Factors
7,Contributing Factors,Environmental,Poison Oak,13,0.565,Environment
8,Contributing Factors,Communications,Insect Stings,20,0.870,Environment
9,Human Factors,Decision Making;Leadership;Risk Assessment,Illness,72,3.130,Human Factors


In [18]:
tab.to_csv("ground_crew_hazards.csv")

In [19]:
docs_per_hazard = {hazard:[doc for year in docs[hazard] for doc in docs[hazard][year]] for hazard in docs}

In [20]:
hazards = []; tracking_num = []

In [21]:
for hazard in docs_per_hazard:
    for num in docs_per_hazard[hazard]:
        hazards.append(hazard)
        tracking_num.append(num)
hazard_docs_df = pd.DataFrame({"Hazard":hazards,
                              "Tracking #": tracking_num})

In [22]:
hazard_docs_df.to_csv(os.path.join("topic_model_results",'hazard_docs.csv'))